In [34]:
import pyspark
from pyspark.sql import SparkSession
from itertools import islice
from pyspark.sql import Row
import pyspark.sql.functions as f

In [2]:
spark = SparkSession.builder.appName('pyspark-training').master('local[2]').getOrCreate()

### Create dataframe using list

In [3]:
row_list = [('John', 26, 'pune'), ('Jessica', 25, 'NY'), ('Robert', 24, 'LA'), ('Adam', 28, 'California')]

In [4]:
col_names = ['name', 'age', 'city']

In [5]:
df_from_list = spark.createDataFrame(row_list, col_names)
df_from_list.show()

+-------+---+----------+
|   name|age|      city|
+-------+---+----------+
|   John| 26|      pune|
|Jessica| 25|        NY|
| Robert| 24|        LA|
|   Adam| 28|California|
+-------+---+----------+



### create dataframe using dict - depricated

In [6]:
row_dict = [{'name': 'John', 'age': 26}, {'name': 'Jessica', 'age': 25},
            {'name': 'Robert', 'age': 24}, {'name': 'Adam', 'age': 28}]
df_from_dict = spark.createDataFrame(row_dict)

In [7]:
df_from_dict.show()

+---+-------+
|age|   name|
+---+-------+
| 26|   John|
| 25|Jessica|
| 24| Robert|
| 28|   Adam|
+---+-------+



### create dataframe using rdd 

In [8]:
from pyspark.sql import Row

In [9]:
Person = Row('name', 'age', 'city')

In [10]:
Person

<Row('name', 'age', 'city')>

In [11]:
rdd = spark.sparkContext.parallelize(row_list)

In [12]:
rdd.collect()

[('John', 26, 'pune'),
 ('Jessica', 25, 'NY'),
 ('Robert', 24, 'LA'),
 ('Adam', 28, 'California')]

In [13]:
person = rdd.map(lambda r : Person(*r))

In [14]:
person.collect()

[Row(name='John', age=26, city='pune'),
 Row(name='Jessica', age=25, city='NY'),
 Row(name='Robert', age=24, city='LA'),
 Row(name='Adam', age=28, city='California')]

In [15]:
df_from_rdd = spark.createDataFrame(person)

In [16]:
df_from_rdd.show()

+-------+---+----------+
|   name|age|      city|
+-------+---+----------+
|   John| 26|      pune|
|Jessica| 25|        NY|
| Robert| 24|        LA|
|   Adam| 28|California|
+-------+---+----------+



### create dataframe by rdd created by reading a textfile

In [17]:
rdd_us_cars = spark.sparkContext.textFile('D:\\data\\us-cars-dataset\\USA_cars_datasets.csv')

In [18]:
rdd_us_cars.take(2)

['row_id,price,brand,model,year,title_status,mileage,color,vin,lot,state,country,condition',
 '0,6300,toyota,cruiser,2008,clean vehicle,274117.0,black,  jtezu11f88k007763,159348797,new jersey, usa,10 days left']

In [19]:
header = rdd_us_cars.first()
rdd_us_cars = rdd_us_cars.filter(lambda line: line != header)

In [20]:
rdd_us_cars.take(5)

['0,6300,toyota,cruiser,2008,clean vehicle,274117.0,black,  jtezu11f88k007763,159348797,new jersey, usa,10 days left',
 '1,2899,ford,se,2011,clean vehicle,190552.0,silver,  2fmdk3gc4bbb02217,166951262,tennessee, usa,6 days left',
 '2,5350,dodge,mpv,2018,clean vehicle,39590.0,silver,  3c4pdcgg5jt346413,167655728,georgia, usa,2 days left',
 '3,25000,ford,door,2014,clean vehicle,64146.0,blue,  1ftfw1et4efc23745,167753855,virginia, usa,22 hours left',
 '4,27700,chevrolet,1500,2018,clean vehicle,6654.0,red,  3gcpcrec2jg473991,167763266,florida, usa,22 hours left']

In [21]:
rdd_us_cars_mapped = rdd_us_cars.map(lambda r : r.split(','))

In [22]:
rdd_us_cars_mapped.take(1)

[['0',
  '6300',
  'toyota',
  'cruiser',
  '2008',
  'clean vehicle',
  '274117.0',
  'black',
  '  jtezu11f88k007763',
  '159348797',
  'new jersey',
  ' usa',
  '10 days left']]

In [23]:
cols = ['row_id', 'price', 'brand', 'model', 'year', 'title_status',
        'mileage', 'color', 'vin', 'lot', 'state', 'country', 'condition']

In [24]:
from pyspark.sql.types import StructField
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType

In [25]:
schema = StructType([StructField(col, StringType(), True) for col in cols])

In [26]:
schema

StructType(List(StructField(row_id,StringType,true),StructField(price,StringType,true),StructField(brand,StringType,true),StructField(model,StringType,true),StructField(year,StringType,true),StructField(title_status,StringType,true),StructField(mileage,StringType,true),StructField(color,StringType,true),StructField(vin,StringType,true),StructField(lot,StringType,true),StructField(state,StringType,true),StructField(country,StringType,true),StructField(condition,StringType,true)))

In [27]:
df_us_cars = spark.createDataFrame(rdd_us_cars_mapped, schema)

In [28]:
df_us_cars.show()

+------+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+
|row_id|price|    brand|  model|year| title_status| mileage| color|                vin|      lot|         state|country|    condition|
+------+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+
|     0| 6300|   toyota|cruiser|2008|clean vehicle|274117.0| black|  jtezu11f88k007763|159348797|    new jersey|    usa| 10 days left|
|     1| 2899|     ford|     se|2011|clean vehicle|190552.0|silver|  2fmdk3gc4bbb02217|166951262|     tennessee|    usa|  6 days left|
|     2| 5350|    dodge|    mpv|2018|clean vehicle| 39590.0|silver|  3c4pdcgg5jt346413|167655728|       georgia|    usa|  2 days left|
|     3|25000|     ford|   door|2014|clean vehicle| 64146.0|  blue|  1ftfw1et4efc23745|167753855|      virginia|    usa|22 hours left|
|     4|27700|chevrolet|   1500|2018|clean vehicle|  66

### Read a file using a csv api

In [29]:
df_us_cars = spark.read.option('header', 'true').csv('D:\\data\\us-cars-dataset\\USA_cars_datasets.csv')

In [30]:
df_us_cars.show()

+------+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+
|row_id|price|    brand|  model|year| title_status| mileage| color|                vin|      lot|         state|country|    condition|
+------+-----+---------+-------+----+-------------+--------+------+-------------------+---------+--------------+-------+-------------+
|     0| 6300|   toyota|cruiser|2008|clean vehicle|274117.0| black|  jtezu11f88k007763|159348797|    new jersey|    usa| 10 days left|
|     1| 2899|     ford|     se|2011|clean vehicle|190552.0|silver|  2fmdk3gc4bbb02217|166951262|     tennessee|    usa|  6 days left|
|     2| 5350|    dodge|    mpv|2018|clean vehicle| 39590.0|silver|  3c4pdcgg5jt346413|167655728|       georgia|    usa|  2 days left|
|     3|25000|     ford|   door|2014|clean vehicle| 64146.0|  blue|  1ftfw1et4efc23745|167753855|      virginia|    usa|22 hours left|
|     4|27700|chevrolet|   1500|2018|clean vehicle|  66

In [31]:
df_us_cars.printSchema()

root
 |-- row_id: string (nullable = true)
 |-- price: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- model: string (nullable = true)
 |-- year: string (nullable = true)
 |-- title_status: string (nullable = true)
 |-- mileage: string (nullable = true)
 |-- color: string (nullable = true)
 |-- vin: string (nullable = true)
 |-- lot: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- condition: string (nullable = true)



In [35]:
price_col =  f.col('price').cast('int')

In [36]:
type(price_col)

pyspark.sql.column.Column

In [37]:
df_us_cars =df_us_cars.withColumn('price', price_col)

In [38]:
df_us_cars.printSchema()

root
 |-- row_id: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- brand: string (nullable = true)
 |-- model: string (nullable = true)
 |-- year: string (nullable = true)
 |-- title_status: string (nullable = true)
 |-- mileage: string (nullable = true)
 |-- color: string (nullable = true)
 |-- vin: string (nullable = true)
 |-- lot: string (nullable = true)
 |-- state: string (nullable = true)
 |-- country: string (nullable = true)
 |-- condition: string (nullable = true)



### Get the dataframe schema

In [39]:
schema1 = df_us_cars.schema

In [40]:
schema1

StructType(List(StructField(row_id,StringType,true),StructField(price,IntegerType,true),StructField(brand,StringType,true),StructField(model,StringType,true),StructField(year,StringType,true),StructField(title_status,StringType,true),StructField(mileage,StringType,true),StructField(color,StringType,true),StructField(vin,StringType,true),StructField(lot,StringType,true),StructField(state,StringType,true),StructField(country,StringType,true),StructField(condition,StringType,true)))

### Get the dataframe columns

In [41]:
cars_cols = df_us_cars.columns

In [42]:
cars_cols

['row_id',
 'price',
 'brand',
 'model',
 'year',
 'title_status',
 'mileage',
 'color',
 'vin',
 'lot',
 'state',
 'country',
 'condition']

### first 

In [43]:
df_us_cars.first()

Row(row_id='0', price=6300, brand='toyota', model='cruiser', year='2008', title_status='clean vehicle', mileage='274117.0', color='black', vin='  jtezu11f88k007763', lot='159348797', state='new jersey', country=' usa', condition='10 days left')

### get n rows

In [44]:
df_us_cars.take(10)

[Row(row_id='0', price=6300, brand='toyota', model='cruiser', year='2008', title_status='clean vehicle', mileage='274117.0', color='black', vin='  jtezu11f88k007763', lot='159348797', state='new jersey', country=' usa', condition='10 days left'),
 Row(row_id='1', price=2899, brand='ford', model='se', year='2011', title_status='clean vehicle', mileage='190552.0', color='silver', vin='  2fmdk3gc4bbb02217', lot='166951262', state='tennessee', country=' usa', condition='6 days left'),
 Row(row_id='2', price=5350, brand='dodge', model='mpv', year='2018', title_status='clean vehicle', mileage='39590.0', color='silver', vin='  3c4pdcgg5jt346413', lot='167655728', state='georgia', country=' usa', condition='2 days left'),
 Row(row_id='3', price=25000, brand='ford', model='door', year='2014', title_status='clean vehicle', mileage='64146.0', color='blue', vin='  1ftfw1et4efc23745', lot='167753855', state='virginia', country=' usa', condition='22 hours left'),
 Row(row_id='4', price=27700, brand=

In [45]:
df_us_cars

DataFrame[row_id: string, price: int, brand: string, model: string, year: string, title_status: string, mileage: string, color: string, vin: string, lot: string, state: string, country: string, condition: string]

### Preparing the loan dataset

In [47]:
df_loans = spark.read.option('header', 'true').option("quote", "\"").option("escape","\\").csv('D:\\data\\loans')

In [48]:
cast_columns = {'loan_id': 'string', 'loan_name': 'string', 'original_language': 'string', 
                'description': 'string', 'description_translated': 'string', 'funded_amount': 'double',
                'loan_amount': 'double', 'status': 'string', 'activity_name': 'string', 'sector_name': 'string',
                'loan_use': 'string', 'country_code': 'string', 'country_name': 'string', 'town_name': 'string',
                'currency_policy': 'string', 'currency_exchange_coverage_rate': 'double', 'currency': 'string',
                'partner_id': 'double', 'posted_time': 'string', 'planned_expiration_time': 'string',
                'disburse_time': 'string', 'raised_time': 'string', 'lender_term': 'double', 'num_lenders_total': 'int',
                'num_journal_entries': 'int', 'num_bulk_entries': 'int', 'tags': 'string', 'borrower_genders': 'string',
                'borrower_pictured': 'string', 'repayment_interval': 'string', 'distribution_model': 'string'}

In [49]:
select_cols = ['loan_id', 'loan_name', 'original_language', 'description', 'description_translated', 'funded_amount',
               'loan_amount', 'status', 'activity_name', 'sector_name', 'loan_use', 'country_code', 'country_name',
               'town_name', 'currency_policy', 'currency_exchange_coverage_rate', 'currency', 'partner_id', 'posted_time',
               'planned_expiration_time', 'disburse_time', 'raised_time', 'lender_term', 'num_lenders_total',
               'num_journal_entries', 'num_bulk_entries', 'tags', 'borrower_genders', 'borrower_pictured',
               'repayment_interval', 'distribution_model']

In [50]:
casted_cols = []
for col_name, col_type in cast_columns.items():
    casted_cols.append(f.col(col_name).cast(col_type))

In [51]:
df_loans_casted = df_loans.select([c for c in casted_cols])

In [52]:
df_loans_ordered = df_loans_casted.select([c for c in select_cols])

In [53]:
df3 = df_loans_ordered.filter('funded_amount is not null')
df4 = df3.filter('loan_amount is not  null')
df5 = df4.filter('num_lenders_total is not  null')
df6 = df5.filter('num_journal_entries is not  null')
df7 = df6.filter('num_bulk_entries is not  null')
df8 = df7.filter('partner_id is not  null')
df9 = df8.filter('lender_term is not  null')
df10 = df9.filter('currency_exchange_coverage_rate is not null')

In [178]:
df10.coalesce(1).write.mode('overwrite').parquet('D:\\data\\loans_pqt')

### Read loans data

In [179]:
df_loans = spark.read.parquet('D:\\data\\loans_pqt')

In [180]:
df_loans.columns

['loan_id',
 'loan_name',
 'original_language',
 'description',
 'description_translated',
 'funded_amount',
 'loan_amount',
 'status',
 'activity_name',
 'sector_name',
 'loan_use',
 'country_code',
 'country_name',
 'town_name',
 'currency_policy',
 'currency_exchange_coverage_rate',
 'currency',
 'partner_id',
 'posted_time',
 'planned_expiration_time',
 'disburse_time',
 'raised_time',
 'lender_term',
 'num_lenders_total',
 'num_journal_entries',
 'num_bulk_entries',
 'tags',
 'borrower_genders',
 'borrower_pictured',
 'repayment_interval',
 'distribution_model']

### print schema

In [57]:
df_loans.printSchema()

root
 |-- loan_id: string (nullable = true)
 |-- loan_name: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- description: string (nullable = true)
 |-- description_translated: string (nullable = true)
 |-- funded_amount: double (nullable = true)
 |-- loan_amount: double (nullable = true)
 |-- status: string (nullable = true)
 |-- activity_name: string (nullable = true)
 |-- sector_name: string (nullable = true)
 |-- loan_use: string (nullable = true)
 |-- country_code: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- town_name: string (nullable = true)
 |-- currency_policy: string (nullable = true)
 |-- currency_exchange_coverage_rate: double (nullable = true)
 |-- currency: string (nullable = true)
 |-- partner_id: double (nullable = true)
 |-- posted_time: string (nullable = true)
 |-- planned_expiration_time: string (nullable = true)
 |-- disburse_time: string (nullable = true)
 |-- raised_time: string (nullable = true)
 |-- len

### show 20 rows

In [58]:
df_loans.show(10, False)

+-------+----------------+-----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### get first row

In [59]:
x = df_loans.first()

In [60]:
x

Row(loan_id='1057058', loan_name='\\tLucy', original_language='English', description='Lucy is a married woman and has been blessed with five children, all of whom attend school. Together with her husband, they live in their own house that has neither electricity nor piped water. Her monthly expenses are school fees.<br /><br />Lucy has operated a fish selling business for over four years, selling from home to restaurant operators, and passersby. She faces a major challenge of seasonality and perishability leading to losses.<br /><br />With the KES 30,000, Lucy wants to purchase bundles of fish for resale. She dreams of expanding her business and establishing another business in the near future.', description_translated='Lucy is a married woman and has been blessed with five children, all of whom attend school. Together with her husband, they live in their own house that has neither electricity nor piped water. Her monthly expenses are school fees.<br /><br />Lucy has operated a fish se

### get n rows

In [61]:
df_loans.take(10)

[Row(loan_id='1057058', loan_name='\\tLucy', original_language='English', description='Lucy is a married woman and has been blessed with five children, all of whom attend school. Together with her husband, they live in their own house that has neither electricity nor piped water. Her monthly expenses are school fees.<br /><br />Lucy has operated a fish selling business for over four years, selling from home to restaurant operators, and passersby. She faces a major challenge of seasonality and perishability leading to losses.<br /><br />With the KES 30,000, Lucy wants to purchase bundles of fish for resale. She dreams of expanding her business and establishing another business in the near future.', description_translated='Lucy is a married woman and has been blessed with five children, all of whom attend school. Together with her husband, they live in their own house that has neither electricity nor piped water. Her monthly expenses are school fees.<br /><br />Lucy has operated a fish s

### different loan status

In [62]:
df_loans.select("status").distinct().show()

+-----------+
|     status|
+-----------+
|   refunded|
|fundRaising|
|    expired|
|     funded|
+-----------+



### filter 

In [63]:
df_loans.filter(df_loans.status == 'funded').count()

1510115

In [64]:
df_loans.filter("status = 'refunded'").count()

5336

In [65]:
df_loans.where("status = 'expired'").count()

77642

### select distinct sector names

In [66]:
df_loans.select('sector_name').distinct().show(truncate=False) # 15 sectors

+--------------+
|sector_name   |
+--------------+
|Services      |
|Wholesale     |
|Education     |
|Food          |
|Entertainment |
|Housing       |
|Health        |
|Clothing      |
|Personal Use  |
|Construction  |
|Agriculture   |
|Arts          |
|Retail        |
|Manufacturing |
|Transportation|
+--------------+



### number of activities in Services sector

In [67]:
df_loans.where("sector_name = 'Services'").select('activity_name').distinct().show()

+-------------------+
|      activity_name|
+-------------------+
|           Services|
|   Machinery Rental|
|Mobile Transactions|
|  Motorcycle Repair|
|          Computers|
|             Energy|
|          Tailoring|
|      Internet Cafe|
|            Laundry|
|        Electrician|
|           Printing|
|     Communications|
|        Call Center|
|        Bookbinding|
|            Cobbler|
|          Utilities|
|              Hotel|
| Electronics Repair|
|    Vehicle Repairs|
|   Air Conditioning|
+-------------------+
only showing top 20 rows



In [68]:
df_loans.where("sector_name = 'Services'").select('activity_name').distinct().count()

39

### number of records with mobile transactions

In [69]:
# 1
df_loans.where("sector_name = 'Services' and activity_name = 'Mobile Transactions'").count()

1624

In [70]:
# 2
df_loans.where((df_loans.sector_name == 'Services') & (df_loans.activity_name == 'Mobile Transactions')).count()

1624

In [71]:
# 3
df_loans.where((f.col('sector_name') == 'Services') & (f.col('activity_name') == 'Mobile Transactions')).count()

1624

### number of records with Beauty Salon and Barber Shop

In [72]:
df_loans.where((f.col('sector_name') == 'Services') & (f.col('activity_name').isin('Beauty Salon', 'Barber Shop'))).count()

19558

### loans with disburse time between 2015-01-01 and 2018-01-01

In [73]:
df_loans.select('disburse_time').show(10, False)

+-----------------------------+
|disburse_time                |
+-----------------------------+
|2016-04-18 07:00:00.000 +0000|
|2012-12-14 08:00:00.000 +0000|
|2011-09-13 07:00:00.000 +0000|
|2014-09-20 07:00:00.000 +0000|
|2014-04-10 07:00:00.000 +0000|
|2020-03-04 08:00:00.000 +0000|
|2012-10-20 07:00:00.000 +0000|
|2018-05-31 07:00:00.000 +0000|
|2018-08-23 07:00:00.000 +0000|
|2013-04-08 07:00:00.000 +0000|
+-----------------------------+
only showing top 10 rows



In [ ]:
# date - yyyy-MM-dd 

In [ ]:
# dd/mm/yyyy - wrong

In [74]:
df_loans.select(f.col('disburse_time').substr(1, 10).alias('disbursed_date').cast('date')).show()

+--------------+
|disbursed_date|
+--------------+
|    2016-04-18|
|    2012-12-14|
|    2011-09-13|
|    2014-09-20|
|    2014-04-10|
|    2020-03-04|
|    2012-10-20|
|    2018-05-31|
|    2018-08-23|
|    2013-04-08|
|    2017-09-14|
|    2013-12-19|
|    2014-12-19|
|    2012-10-16|
|    2019-04-30|
|    2014-02-20|
|    2019-12-30|
|    2013-06-20|
|    2018-12-26|
|    2014-11-28|
+--------------+
only showing top 20 rows



In [75]:
df_loans.filter(f.col('disburse_time').substr(1, 10).cast('date').between('2015-01-01', '2018-01-01'))\
.select('loan_id', 'loan_name', 'disburse_time').show(truncate=False)

+-------+---------------+-----------------------------+
|loan_id|loan_name      |disburse_time                |
+-------+---------------+-----------------------------+
|1057058|\tLucy         |2016-04-18 07:00:00.000 +0000|
|1383778|Yesica Marilú  |2017-09-14 07:00:00.000 +0000|
|835424 |Vilma          |2015-01-26 08:00:00.000 +0000|
|1296242|Mulyati S      |2017-05-08 07:00:00.000 +0000|
|1233188|Nida           |2017-01-20 08:00:00.000 +0000|
|950357 |Bernaditha     |2015-08-25 07:00:00.000 +0000|
|1272007|Rebecca        |2017-03-06 08:00:00.000 +0000|
|1316761|Glenda         |2017-05-24 07:00:00.000 +0000|
|1124820|ROSALINA       |2016-07-14 07:00:00.000 +0000|
|1306337|Nancy          |2017-05-05 07:00:00.000 +0000|
|1033537|Susan          |2016-02-05 08:00:00.000 +0000|
|871235 |Durdona        |2015-03-17 07:00:00.000 +0000|
|987778 |Thanh          |2015-10-31 07:00:00.000 +0000|
|938334 |Rafael de Jesus|2015-08-11 07:00:00.000 +0000|
|1421780|Leonara        |2017-10-27 07:00:00.000

### add a new column

In [181]:
df_loans2 = df_loans.withColumn('disbursed_date', f.col('disburse_time').substr(1, 10).cast('date'))

In [77]:
df_loans2

DataFrame[loan_id: string, loan_name: string, original_language: string, description: string, description_translated: string, funded_amount: double, loan_amount: double, status: string, activity_name: string, sector_name: string, loan_use: string, country_code: string, country_name: string, town_name: string, currency_policy: string, currency_exchange_coverage_rate: double, currency: string, partner_id: double, posted_time: string, planned_expiration_time: string, disburse_time: string, raised_time: string, lender_term: double, num_lenders_total: int, num_journal_entries: int, num_bulk_entries: int, tags: string, borrower_genders: string, borrower_pictured: string, repayment_interval: string, distribution_model: string, disbursed_date: date]

In [78]:
df_loans2.withColumn('flag', f.lit('Y')).show(5) #literal

+-------+---------+-----------------+--------------------+----------------------+-------------+-----------+------+--------------------+-----------+--------------------+------------+------------+------------+---------------+-------------------------------+--------+----------+--------------------+-----------------------+--------------------+--------------------+-----------+-----------------+-------------------+----------------+--------------------+----------------+-----------------+------------------+------------------+--------------+----+
|loan_id|loan_name|original_language|         description|description_translated|funded_amount|loan_amount|status|       activity_name|sector_name|            loan_use|country_code|country_name|   town_name|currency_policy|currency_exchange_coverage_rate|currency|partner_id|         posted_time|planned_expiration_time|       disburse_time|         raised_time|lender_term|num_lenders_total|num_journal_entries|num_bulk_entries|                tags|borrow

In [182]:
df_loans2.withColumn('created_at', f.lit('2022-01-16'))\
.write.mode('overwrite')\
.partitionBy('created_at').parquet('D:\\data\\loans_pqt_part')

### groupby

In [184]:
df_loans.select('status').distinct().show()

+-----------+
|     status|
+-----------+
|   refunded|
|fundRaising|
|    expired|
|     funded|
+-----------+



In [185]:
df_loans.groupby('status').count().show()

+-----------+-------+
|     status|  count|
+-----------+-------+
|   refunded|   5336|
|fundRaising|   2730|
|    expired|  77642|
|     funded|1510115|
+-----------+-------+



### number of loans in different countries

In [186]:
df_loans.groupby('country_name').count().orderBy(f.col('count').desc()).show()

+------------+------+
|country_name| count|
+------------+------+
| Philippines|346572|
|       Kenya|229053|
|      Uganda| 72616|
|    Cambodia| 70748|
| El Salvador| 63578|
|        Peru| 59567|
|  Tajikistan| 55386|
|    Pakistan| 55007|
|    Colombia| 46720|
|   Nicaragua| 29653|
|     Vietnam| 27675|
|       India| 25224|
|    Paraguay| 24612|
|     Nigeria| 22833|
|      Rwanda| 22700|
|     Liberia| 21887|
|        Togo| 21771|
|       Ghana| 20812|
|    Honduras| 20384|
|  Madagascar| 20072|
+------------+------+
only showing top 20 rows



In [ ]:
df_loans.groupBy('status').max('')

#### funded_amount - The amount disbursed by Kiva to the field agent(USD)
#### loan_amount - The amount disbursed by the field agent to the borrower(USD)

### max loan amount for specific country

In [189]:
df_max_loan_in_country = df_loans.groupby('country_name').agg({'loan_amount': 'max'})\
                                 .withColumnRenamed('max(loan_amount)', 'max_loan_amt')

In [190]:
df_max_loan_in_country.show()

+------------+------------+
|country_name|max_loan_amt|
+------------+------------+
|    Paraguay|    355225.0|
|       Yemen|     13975.0|
|     Senegal|      6200.0|
| Philippines|    100000.0|
|       Tonga|      5600.0|
|        Fiji|      3975.0|
|      Turkey|     12375.0|
|      Malawi|     50000.0|
|        Iraq|      3000.0|
|    Cambodia|     50000.0|
|      Jordan|      4575.0|
|      Rwanda|    408375.0|
|      Kosovo|      2975.0|
|   Sri Lanka|       875.0|
|        Togo|      5550.0|
|     Ecuador|     10000.0|
|     Lesotho|      2400.0|
|     Albania|     50000.0|
|  Madagascar|     50000.0|
|       Ghana|     50000.0|
+------------+------------+
only showing top 20 rows



### min funded amount

In [82]:
df_loans.groupby('country_name').min('funded_amount').show()

+------------+------------------+
|country_name|min(funded_amount)|
+------------+------------------+
|    Paraguay|               0.0|
|       Yemen|              50.0|
|     Senegal|               0.0|
| Philippines|               0.0|
|       Tonga|               0.0|
|        Fiji|             100.0|
|      Turkey|               0.0|
|      Malawi|              50.0|
|        Iraq|            1200.0|
|    Cambodia|               0.0|
|      Jordan|               0.0|
|      Rwanda|               0.0|
|      Kosovo|              25.0|
|   Sri Lanka|             100.0|
|        Togo|               0.0|
|     Ecuador|              25.0|
|     Lesotho|               0.0|
|     Albania|               0.0|
|  Madagascar|               0.0|
|       Ghana|              25.0|
+------------+------------------+
only showing top 20 rows



### avg funded amount

In [191]:
df_loans.groupby('country_name').agg(f.avg('funded_amount').alias('avg_funded_amt'))\
        .orderBy(f.col('avg_funded_amt').asc()).show()

+------------+------------------+
|country_name|    avg_funded_amt|
+------------+------------------+
|  Madagascar|247.33334994021521|
|     Nigeria| 264.9465685630447|
|   Sri Lanka| 283.3333333333333|
|     Liberia| 329.4786859779778|
| Philippines|  343.405699248641|
|     Lesotho| 364.0006763611769|
| South Sudan| 373.2387569967188|
|      Turkey| 403.3796155019835|
|        Togo|409.09351890129074|
|       Nepal| 413.9229636898921|
|       Kenya|416.28138465769933|
|    Cameroon|425.75014153613887|
|       India|  490.630946717412|
|    Pakistan|  515.081716872398|
|    Colombia| 526.3511344178082|
|   Indonesia| 549.8694708435996|
|    Cambodia| 583.0680725956918|
|  Mozambique|  591.769334299617|
| El Salvador| 593.8319859070748|
|      Uganda| 639.2702710146524|
+------------+------------------+
only showing top 20 rows



### mean funded amount

In [84]:
df_loans.groupby('country_name').mean('funded_amount').show()

+------------+------------------+
|country_name|avg(funded_amount)|
+------------+------------------+
|    Paraguay|2438.5064196326994|
|       Yemen|  792.452942578032|
|     Senegal| 2090.940918130593|
| Philippines|  343.405699248641|
|       Tonga|1155.3858520900321|
|        Fiji| 735.0470588235294|
|      Turkey| 403.3796155019835|
|      Malawi|1448.4389782403027|
|        Iraq| 2458.119658119658|
|    Cambodia| 583.0680725956918|
|      Jordan|1006.9950357810586|
|      Rwanda|2015.4438325991189|
|      Kosovo|1277.3489932885907|
|   Sri Lanka| 283.3333333333333|
|        Togo|409.09351890129074|
|     Ecuador| 943.1407015326275|
|     Lesotho| 364.0006763611769|
|     Albania|1388.3080077182826|
|  Madagascar|247.33334994021521|
|       Ghana| 706.9491158946762|
+------------+------------------+
only showing top 20 rows



### sum funded amount

In [85]:
df_loans.groupby('country_name').sum('funded_amount').show()

+------------+------------------+
|country_name|sum(funded_amount)|
+------------+------------------+
|    Paraguay|        6.001652E7|
|       Yemen|         3325925.0|
|     Senegal|       2.5233475E7|
| Philippines|        1.190148E8|
|       Tonga|         1437300.0|
|        Fiji|         1561975.0|
|      Turkey|         1321875.0|
|      Malawi|         4593000.0|
|        Iraq|          287600.0|
|    Cambodia|         4.12509E7|
|      Jordan|         1.56195E7|
|      Rwanda|       4.5750575E7|
|      Kosovo|         3806500.0|
|   Sri Lanka|           74800.0|
|        Togo|         8906375.0|
|     Ecuador|       1.1938275E7|
|     Lesotho|         1076350.0|
|     Albania|         5755925.0|
|  Madagascar|         4964475.0|
|       Ghana|       1.4713025E7|
+------------+------------------+
only showing top 20 rows



### pivot

In [192]:
df_loans.select('country_name', 'sector_name', 'funded_amount').show()

+------------+------------+-------------+
|country_name| sector_name|funded_amount|
+------------+------------+-------------+
|       Kenya|        Food|        300.0|
|       Kenya|      Retail|        300.0|
| Philippines|    Services|        475.0|
|    Zimbabwe|        Food|        500.0|
|     Liberia|      Retail|        400.0|
|      Turkey|        Arts|        225.0|
|    Cambodia| Agriculture|        625.0|
|    Colombia|        Food|        850.0|
|      Uganda|        Food|        550.0|
|Sierra Leone|        Food|        125.0|
| El Salvador|      Retail|       1000.0|
|       Samoa| Agriculture|        450.0|
|    Cambodia|Personal Use|       1450.0|
|      Jordan|    Clothing|        850.0|
| Philippines|      Retail|        575.0|
| Philippines|      Retail|        450.0|
|    Paraguay|      Retail|       3275.0|
| Philippines| Agriculture|        300.0|
|   Indonesia|     Housing|        450.0|
| Philippines|        Food|        250.0|
+------------+------------+-------

In [193]:
df_loans_pivot = df_loans.groupby('country_name').pivot('sector_name').sum('funded_amount')

### fillna

In [87]:
df_loans_pivot.fillna(0).show()

+------------+-----------+---------+---------+------------+---------+-------------+-----------+---------+---------+-------------+------------+-----------+---------+--------------+---------+
|country_name|Agriculture|     Arts| Clothing|Construction|Education|Entertainment|       Food|   Health|  Housing|Manufacturing|Personal Use|     Retail| Services|Transportation|Wholesale|
+------------+-----------+---------+---------+------------+---------+-------------+-----------+---------+---------+-------------+------------+-----------+---------+--------------+---------+
|    Paraguay|  2689525.0|1010800.0|8049650.0|    576200.0|4566975.0|       2650.0|1.8356795E7| 879900.0|  83850.0|      85400.0|     93300.0|1.4574425E7|8923075.0|      109950.0|  14025.0|
|       Yemen|    91425.0|   7800.0| 255200.0|    192225.0|   6675.0|          0.0|   509625.0|  49825.0| 578950.0|      16700.0|    612125.0|   489500.0| 284200.0|      227625.0|   4050.0|
|     Senegal|  5858800.0| 112375.0| 772025.0|    

### rename column

In [202]:
df_max_loan_in_country_renamed = df_max_loan_in_country.withColumnRenamed('max(loan_amount)', 'max_loan_amt')

In [203]:
df_max_loan_in_country_renamed

DataFrame[country_name: string, max_loan_amt: double]

### fetch paraguay max loan

In [201]:
df_max_loan_in_country_renamed.filter("country_name = 'Paraguay'").select('max_loan_amt').collect()[0]['max_loan_amt']

355225.0

In [204]:
paraguay_max_loan = df_max_loan_in_country_renamed.filter("country_name = 'Paraguay'").select('max_loan_amt').collect()[0][0]

In [205]:
paraguay_max_loan

355225.0

### details of max loan id

In [206]:
df_loans.filter('loan_amount = {}'.format(paraguay_max_loan)).show()

+-------+-------------------+-----------------+--------------------+----------------------+-------------+-----------+------+-------------+-----------+--------------------+------------+------------+---------+---------------+-------------------------------+--------+----------+--------------------+-----------------------+--------------------+--------------------+-----------+-----------------+-------------------+----------------+--------------------+----------------+-----------------+------------------+------------------+
|loan_id|          loan_name|original_language|         description|description_translated|funded_amount|loan_amount|status|activity_name|sector_name|            loan_use|country_code|country_name|town_name|currency_policy|currency_exchange_coverage_rate|currency|partner_id|         posted_time|planned_expiration_time|       disburse_time|         raised_time|lender_term|num_lenders_total|num_journal_entries|num_bulk_entries|                tags|borrower_genders|borrower_

In [93]:
df_loans.where("loan_id in (638631, 640322)").show()

+-------+---------------+-----------------+--------------------+----------------------+-------------+-----------+------+-------------+-----------+--------------------+------------+------------+--------------------+---------------+-------------------------------+--------+----------+--------------------+-----------------------+--------------------+--------------------+-----------+-----------------+-------------------+----------------+--------------------+--------------------+--------------------+------------------+------------------+
|loan_id|      loan_name|original_language|         description|description_translated|funded_amount|loan_amount|status|activity_name|sector_name|            loan_use|country_code|country_name|           town_name|currency_policy|currency_exchange_coverage_rate|currency|partner_id|         posted_time|planned_expiration_time|       disburse_time|         raised_time|lender_term|num_lenders_total|num_journal_entries|num_bulk_entries|                tags|    b

### add a column mentioning number of borrowers in a loan id

In [207]:
df_loans.select('loan_id', 'borrower_genders').show(truncate=False)

+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|loan_id|borrower_genders                                                                                                                                              |
+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1057058|female                                                                                                                                                        |
|509472 |male                                                                                                                                                          |
|344324 |female                                                                                                                                            

In [209]:
df_loans.withColumn('num_borrowers',f.size(f.split('borrower_genders', ','))).select('loan_id', 'num_borrowers').show()

+-------+-------------+
|loan_id|num_borrowers|
+-------+-------------+
|1057058|            1|
| 509472|            1|
| 344324|            1|
| 768521|            1|
| 702112|            1|
|1951531|            1|
| 497810|            2|
|1560103|            1|
|1604550|            1|
| 557284|            1|
|1383778|            1|
| 657788|            1|
| 828660|           14|
| 492275|            1|
|1765926|            1|
| 683605|            1|
|1920651|           20|
| 579961|            1|
|1690660|            1|
| 812083|            1|
+-------+-------------+
only showing top 20 rows



### create temp view 

In [215]:
df_loans.createOrReplaceTempView('loans')

### run sql query

In [216]:
spark.sql("select * from loans where status = 'funded'").show()

+-------+--------------------+-----------------+--------------------+----------------------+-------------+-----------+------+--------------------+------------+--------------------+------------+------------+--------------------+---------------+-------------------------------+--------+----------+--------------------+-----------------------+--------------------+--------------------+-----------+-----------------+-------------------+----------------+--------------------+--------------------+--------------------+------------------+------------------+
|loan_id|           loan_name|original_language|         description|description_translated|funded_amount|loan_amount|status|       activity_name| sector_name|            loan_use|country_code|country_name|           town_name|currency_policy|currency_exchange_coverage_rate|currency|partner_id|         posted_time|planned_expiration_time|       disburse_time|         raised_time|lender_term|num_lenders_total|num_journal_entries|num_bulk_entries|

### drop temp view

In [213]:
spark.catalog.dropTempView('loans')

### drop a column

In [217]:
df_loans_dropped = df_loans.drop('description')

### list of columns

In [219]:
df_loans.columns

['loan_id',
 'loan_name',
 'original_language',
 'description',
 'description_translated',
 'funded_amount',
 'loan_amount',
 'status',
 'activity_name',
 'sector_name',
 'loan_use',
 'country_code',
 'country_name',
 'town_name',
 'currency_policy',
 'currency_exchange_coverage_rate',
 'currency',
 'partner_id',
 'posted_time',
 'planned_expiration_time',
 'disburse_time',
 'raised_time',
 'lender_term',
 'num_lenders_total',
 'num_journal_entries',
 'num_bulk_entries',
 'tags',
 'borrower_genders',
 'borrower_pictured',
 'repayment_interval',
 'distribution_model']

In [218]:
df_loans_dropped.columns

['loan_id',
 'loan_name',
 'original_language',
 'description_translated',
 'funded_amount',
 'loan_amount',
 'status',
 'activity_name',
 'sector_name',
 'loan_use',
 'country_code',
 'country_name',
 'town_name',
 'currency_policy',
 'currency_exchange_coverage_rate',
 'currency',
 'partner_id',
 'posted_time',
 'planned_expiration_time',
 'disburse_time',
 'raised_time',
 'lender_term',
 'num_lenders_total',
 'num_journal_entries',
 'num_bulk_entries',
 'tags',
 'borrower_genders',
 'borrower_pictured',
 'repayment_interval',
 'distribution_model']

### case when

### add a column with repayment regularity. For irregular repayment mark 0 else 1

In [220]:
df_loans.select('repayment_interval').distinct().show()

+------------------+
|repayment_interval|
+------------------+
|           monthly|
|         irregular|
|            bullet|
|           Ninfa""|
|             KHADY|
|           KOUMBA |
|            DIAMY |
|  Iliben Lalabhai"|
+------------------+



In [222]:
df_loans.withColumn('repayment_flag', 
                    f.when(f.col('repayment_interval').isin('monthly', 'bullet'), 1 ).otherwise(0))\
        .select('repayment_flag').show()

+--------------+
|repayment_flag|
+--------------+
|             1|
|             1|
|             1|
|             1|
|             1|
|             1|
|             0|
|             1|
|             1|
|             1|
|             1|
|             1|
|             1|
|             1|
|             1|
|             1|
|             1|
|             1|
|             1|
|             1|
+--------------+
only showing top 20 rows



In [151]:
df_loans.select('status').distinct().show()

+-----------+
|     status|
+-----------+
|   refunded|
|fundRaising|
|    expired|
|     funded|
+-----------+



In [221]:
df_loans.withColumn('funded', 
                    f.when(f.col('status').__eq__('funded') | f.col('status').__eq__('refunded'), f.lit('Y'))\
                    .otherwise('N')).select('status', 'funded').distinct().show()

+-----------+------+
|     status|funded|
+-----------+------+
|    expired|     N|
|fundRaising|     N|
|     funded|     Y|
|   refunded|     Y|
+-----------+------+



### Join - inner, left, right, full, cross, leftanti

In [225]:
df_loans.select('loan_id').distinct().count()

1595823

In [223]:
df_loan_themes = spark.read.option('header', 'true').csv('D:\\data\\kiva\\loan_theme_ids.csv')\
                .withColumnRenamed('id', 'loan_id')\
                .withColumnRenamed('Loan Theme ID', 'loan_theme_id')\
                .withColumnRenamed('Loan Theme Type', 'loan_theme_type')\
                .withColumnRenamed('Partner ID', 'partner_id')\
                .drop('partner_id')

In [226]:
df_loan_themes.show(10, False)

+-------+---------------+-----------------------+
|loan_id|loan_theme_id  |loan_theme_type        |
+-------+---------------+-----------------------+
|638631 |a1050000000skGl|General                |
|640322 |a1050000000skGl|General                |
|641006 |a1050000002X1ij|Higher Education       |
|641019 |a1050000002X1ij|Higher Education       |
|641594 |a1050000002VbsW|Subsistence Agriculture|
|642256 |a1050000000T3oX|Extreme Poverty        |
|642311 |a1050000000snTK|General                |
|642353 |a1050000000T3oX|Extreme Poverty        |
|642386 |a1050000000wf0f|General                |
|642429 |a1050000000wezJ|General                |
+-------+---------------+-----------------------+
only showing top 10 rows



### inner join - common id within both

In [103]:
df_loan_themes.select('loan_id').distinct().count()

779092

In [104]:
df_loans.join(df_loan_themes, ['loan_id'] ,'inner').count()

704046

In [227]:
df_loans.join(df_loan_themes, ['loan_id'] ,'inner').show()

+-------+----------------+-----------------+--------------------+----------------------+-------------+-----------+------+--------------------+--------------+--------------------+------------+------------+--------------------+---------------+-------------------------------+--------+----------+--------------------+-----------------------+--------------------+--------------------+-----------+-----------------+-------------------+----------------+--------------------+--------------------+--------------------+------------------+------------------+---------------+--------------------+
|loan_id|       loan_name|original_language|         description|description_translated|funded_amount|loan_amount|status|       activity_name|   sector_name|            loan_use|country_code|country_name|           town_name|currency_policy|currency_exchange_coverage_rate|currency|partner_id|         posted_time|planned_expiration_time|       disburse_time|         raised_time|lender_term|num_lenders_total|num_

### left join - everything from left

In [105]:
df_loan_w_theme = df_loans.join(df_loan_themes, 'loan_id', 'left')

In [106]:
df_loan_w_theme.count()

1595823

In [107]:
df_loan_w_theme.where('loan_theme_type is null').count()

891777

### right join - everything from right

In [108]:
df_loan_w_theme_right = df_loans.join(df_loan_themes, ['loan_id'], 'right')

In [109]:
df_loan_w_theme_right.count()

779092

### left anti - not in right

In [110]:
df_loan_w_theme_left_anti = df_loans.join(df_loan_themes, ['loan_id'], 'leftanti')

In [111]:
df_loan_w_theme_left_anti.count()

891777

### full - all from both the sides

In [228]:
df_loan_w_theme_full = df_loans.join(df_loan_themes, ['loan_id'], 'full')

In [113]:
df_loan_w_theme_full.count()

1670869

### cross - m X n

In [229]:
df_common_cat = spark.createDataFrame([('beauty',), ('healthcare',), ('cosmetics',)], ['product_category'])

In [230]:
df_common_cat.show()

+----------------+
|product_category|
+----------------+
|          beauty|
|      healthcare|
|       cosmetics|
+----------------+



In [231]:
df_products = spark.createDataFrame([(1, 'face_wash'), (2, 'body_wash'), (3, 'cream'),
                                     (4, 'body_lotion'), (5, 'hair_oil')], ['id', 'product'])

In [232]:
df_products.show()

+---+-----------+
| id|    product|
+---+-----------+
|  1|  face_wash|
|  2|  body_wash|
|  3|      cream|
|  4|body_lotion|
|  5|   hair_oil|
+---+-----------+



In [233]:
df_prod_cat = df_products.crossJoin(df_common_cat)

In [234]:
df_prod_cat.show()

+---+-----------+----------------+
| id|    product|product_category|
+---+-----------+----------------+
|  1|  face_wash|          beauty|
|  2|  body_wash|          beauty|
|  1|  face_wash|      healthcare|
|  1|  face_wash|       cosmetics|
|  2|  body_wash|      healthcare|
|  2|  body_wash|       cosmetics|
|  3|      cream|          beauty|
|  4|body_lotion|          beauty|
|  5|   hair_oil|          beauty|
|  3|      cream|      healthcare|
|  3|      cream|       cosmetics|
|  4|body_lotion|      healthcare|
|  4|body_lotion|       cosmetics|
|  5|   hair_oil|      healthcare|
|  5|   hair_oil|       cosmetics|
+---+-----------+----------------+



### different tags

In [235]:
df_loans.select('tags').distinct().show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tags                                                                                                                                                                                                                |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|#Parent, #Repeat Borrower, #Woman-Owned Business                                                                                                                                                                    |
|#Single, #Animals, user_favorite                                                                                                           

### convert dataframe to rdd and calculate tag count
### some records doesn't have hashtags. Fill those records with ''
### get distinct tag counts

In [236]:
tags_w_count = df_loans.select('tags').fillna('').rdd\
                       .map(lambda r : r[0].replace('#', '').split(','))\
                       .flatMap(lambda tag : tag).distinct().count()

In [237]:
tags_w_count

85

### use rdd map, flatmap, and reduceByKey for calculating the number of tags

In [126]:
tags_w_count = df_loans.select('tags').fillna('').rdd.map(lambda r : r[0].replace('#', '').split(','))\
.flatMap(lambda tag : tag).map(lambda tag : tag.strip()).map(lambda tag : (tag, 1)).reduceByKey(lambda x,y : x+y).collect()

### print tags with count

In [127]:
for tag in tags_w_count:
    print(tag)

('', 598592)
('Repair Renew Replace', 33509)
('Inspiring Story', 4917)
('Orphan', 968)
('Post-disbursed', 3397)
('Parent', 407990)
('Schooling', 122143)
('user_favorite', 799424)
('Vegan', 99166)
('Interesting Photo', 6642)
('Refugee', 7372)
('cow', 2)
('Fabrics', 49821)
('Woman-Owned Business', 392456)
('Health and Sanitation', 86181)
('Repeat Borrower', 227790)
('Technology', 65489)
('Job Creator', 18938)
('Team Guys Holding Fish', 238)
('Married', 132)
('US immigrant', 13)
('reserved_crisis_support_loan', 7)
('Biz Durable Asset', 80637)
('Supporting Family', 40896)
('Unique', 7312)
('Widowed', 24249)
('Female Education', 12809)
('Single', 33934)
('Tourism', 258)
('Powerful Story', 52)
('Elderly', 157643)
('US Black-Owned Business', 4)
('Sustainable Ag', 32116)
('Low-profit FP', 8653)
('First Loan', 41385)
('Eco-friendly', 116770)
('Animals', 126350)
('volunteer_pick', 55369)
('Hidden Gem', 1773)
('beauty', 2)
('Single Parent', 42414)
('Trees', 15042)
('volunteer_like', 32446)
('Lati

### compute basic statistics for numeric and string column

In [128]:
df_loans.describe().show()

+-------+------------------+--------------------+-----------------+--------------------+----------------------+------------------+------------------+--------+-------------+-----------+--------------------+------------+------------+--------------------+---------------+-------------------------------+--------+------------------+--------------------+-----------------------+--------------------+--------------------+-----------------+------------------+-------------------+-------------------+--------------------+--------------------+--------------------+------------------+------------------+
|summary|           loan_id|           loan_name|original_language|         description|description_translated|     funded_amount|       loan_amount|  status|activity_name|sector_name|            loan_use|country_code|country_name|           town_name|currency_policy|currency_exchange_coverage_rate|currency|        partner_id|         posted_time|planned_expiration_time|       disburse_time|         rai

In [129]:
df_loans.summary().show()

+-------+------------------+--------------------+-----------------+--------------------+----------------------+------------------+------------------+--------+-------------+-----------+--------------------+------------+------------+--------------------+---------------+-------------------------------+--------+------------------+--------------------+-----------------------+--------------------+--------------------+-----------------+------------------+-------------------+-------------------+--------------------+--------------------+--------------------+------------------+------------------+
|summary|           loan_id|           loan_name|original_language|         description|description_translated|     funded_amount|       loan_amount|  status|activity_name|sector_name|            loan_use|country_code|country_name|           town_name|currency_policy|currency_exchange_coverage_rate|currency|        partner_id|         posted_time|planned_expiration_time|       disburse_time|         rai

### explode

In [133]:
from pyspark.sql import Row

In [134]:
eDF = spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])

In [238]:
eDF.printSchema()

root
 |-- a: long (nullable = true)
 |-- intlist: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- mapfield: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [137]:
eDF.show()

+---+---------+--------+
|  a|  intlist|mapfield|
+---+---------+--------+
|  1|[1, 2, 3]|{a -> b}|
+---+---------+--------+



In [139]:
eDF.select(f.col('a'), f.explode(eDF.intlist).alias("anInt")).show()

+---+-----+
|  a|anInt|
+---+-----+
|  1|    1|
|  1|    2|
|  1|    3|
+---+-----+



In [141]:
eDF.select(f.col('a'), f.explode(eDF.mapfield).alias("key", "value")).show()

+---+---+-----+
|  a|key|value|
+---+---+-----+
|  1|  a|    b|
+---+---+-----+



### concat

In [158]:
df = spark.createDataFrame([('abcd','123')], ['s', 'd'])
df.select(f.concat(df.s, df.d).alias('s')).show()

+-------+
|      s|
+-------+
|abcd123|
+-------+



### concat_ws

In [160]:
df = spark.createDataFrame([('abcd','123')], ['s', 'd'])
df.select(f.concat_ws('-', df.s, df.d).alias('s')).show()

+--------+
|       s|
+--------+
|abcd-123|
+--------+



### date operations

In [239]:
df = spark.createDataFrame([('2022-01-16 11:28:11',)], ['ts']).withColumn('ts', f.col('ts').cast('timestamp'))

In [240]:
df.printSchema()

root
 |-- ts: timestamp (nullable = true)



In [165]:
df.show()

+-------------------+
|                 ts|
+-------------------+
|2022-01-16 11:28:11|
+-------------------+



In [167]:
df.withColumn('dt', f.to_date('ts')).show()

+-------------------+----------+
|                 ts|        dt|
+-------------------+----------+
|2022-01-16 11:28:11|2022-01-16|
+-------------------+----------+



In [168]:
df.withColumn('month', f.month('ts')).show()

+-------------------+-----+
|                 ts|month|
+-------------------+-----+
|2022-01-16 11:28:11|    1|
+-------------------+-----+



In [169]:
df.withColumn('month', f.year('ts')).show()

+-------------------+-----+
|                 ts|month|
+-------------------+-----+
|2022-01-16 11:28:11| 2022|
+-------------------+-----+



In [171]:
df.withColumn('weekofyear', f.weekofyear('ts')).show()

+-------------------+----------+
|                 ts|weekofyear|
+-------------------+----------+
|2022-01-16 11:28:11|         2|
+-------------------+----------+



In [173]:
df.withColumn('dayofweek', f.dayofweek('ts')).show()

+-------------------+---------+
|                 ts|dayofweek|
+-------------------+---------+
|2022-01-16 11:28:11|        1|
+-------------------+---------+



In [174]:
df.withColumn('dayofmonth', f.dayofmonth('ts')).show()

+-------------------+----------+
|                 ts|dayofmonth|
+-------------------+----------+
|2022-01-16 11:28:11|        16|
+-------------------+----------+



In [176]:
df.withColumn('dayofyear', f.dayofyear('ts')).show()

+-------------------+---------+
|                 ts|dayofyear|
+-------------------+---------+
|2022-01-16 11:28:11|       16|
+-------------------+---------+

